## Spatial Query of GLA06 Data for Greenland

Set up a grid cell query for Jakobshavn.

In [2]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from osgeo import osr, gdal, ogr

# try to display same using basemap
from mpl_toolkits.basemap import Basemap


# an approximate box in lat-lon
jak_lon_ll = -52.
jak_lat_ll = 68.2
jak_lon_ur = -45.
jak_lat_ur = 70.

# here's the approx. bounding box in PS
jak_psx_ll = -291000.
jak_psy_ll = -2371000.
jak_psx_ur = -500.
jak_psy_ur = -2189000.
jak_bbox=(jak_psx_ll, jak_psy_ll, jak_psx_ur, jak_psy_ur)

psx_ll, psy_ll, psx_ur, psy_ur = jak_bbox

          
coords = "{0} {1},".format(psx_ll, psy_ll) + \
    "{0} {1},".format(psx_ll, psy_ur) + \
    "{0} {1},".format(psx_ur, psy_ur) + \
    "{0} {1},".format(psx_ur, psy_ll) + \
    "{0} {1}".format(psx_ll, psy_ll)
    
bbox = "ST_MakePolygon(ST_GeomFromText('LINESTRING({0})',3413))".format(coords)
print bbox

ST_MakePolygon(ST_GeomFromText('LINESTRING(-291000.0 -2371000.0,-291000.0 -2189000.0,-500.0 -2189000.0,-500.0 -2371000.0,-291000.0 -2371000.0)',3413))


Given this polygon the SQL query to extract the point would be something like the following:

    select count(*) from gla06_034_grn where
    ST_Contains(ST_MakePolygon(ST_GeomFromText('LINESTRING(-291000.0 -2371000.0,
            -291000.0 -2189000.0,-500.0 -2189000.0,-500.0 -2371000.0,
            -291000.0 -2371000.0)',3413)),
    the_geom);
    
The `gla06_034_grn` table has about 45 million rows.  This query took about .7 seconds and returned 221,775 rows.  

In [4]:
# set up a grid 
gridsize = 1000.
jak_xvals = np.arange(jak_psx_ll + gridsize/2., jak_psx_ur - gridsize/2., gridsize)
jak_yvals = np.arange(jak_psy_ll + gridsize/2., jak_psy_ur - gridsize/2., gridsize)
nx = len(jak_xvals)
ny = len(jak_yvals)
print nx, ny

290 181


In [5]:
EPSG_CODE = 3413
def gridcell2poly(cellcntr, cellsize):
    """
    Create a postGIS WKT Polygon geometry for a grid cell.  The cell
    center is given as a list of X and Y values.  The cell size is a
    list of dX and dY values.  The returned value is a string.

    Here's an example WKT Polygon string:

    ST_MakePolygon(
      ST_GeomFromText(
      'LINESTRING(75.15 29.53,77 29,77.6 29.5, 75.15 29.53)',
      4326));

    (I've added line breaks to make this readable.)
    """
    xcntr, ycntr = cellcntr
    deltax, deltay = cellsize

    points = [[xcntr-deltax, ycntr+deltay],
              [xcntr+deltax, ycntr+deltay],
              [xcntr+deltax, ycntr-deltay],
              [xcntr-deltax, ycntr-deltay],
              [xcntr-deltax, ycntr+deltay]]

    coordlist = ["{0} {1}".format(x[0], x[1]) for x in points]
    coordstr = ",".join(coordlist)
    polygeom = "ST_MakePolygon(ST_GeomFromText" + \
               "('LINESTRING({0})',{1}))".format( coordstr,
                                                  EPSG_CODE)

    return polygeom

In [11]:
import psycopg2
import numpy as np

DBNAME = 'icedb'
DBHOST = 'icebridge.sr.unh.edu'
DBUSER = 'nobody'
DBPORT = '5432'
EPSG_CODE = 3413

# make database connection
conn_str = 'host={0} dbname={1} user={2}'.format(DBHOST, DBNAME, DBUSER)
try:
    conn = psycopg2.connect(conn_str)
except:
    print "connection to database failed"

In [19]:
# set up output array for results
outarr = np.zeros([ny,nx],np.int)
print outarr.shape

# set up cursor for query
mycur = conn.cursor()


# loop over x and y
for psy in jak_yvals:
    for psx in jak_xvals:
        bbox = gridcell2poly([psx, psy], [gridsize, gridsize])
        mycur = conn.cursor()
        query = "select count(*) from gla06_034_grn where ST_Contains({0},the_geom);"
        query = query.format(bbox)
        mycur.execute(query)
        results = mycur.fetchall()
        print results[0][0]
        break
    break
    
mycur.close()

(181, 290)
0


In [ ]:
# set up query
